# deep research (agent driven)

In [3]:
"""Prompt templates and tool descriptions for the research deepagent."""

RESEARCH_WORKFLOW_INSTRUCTIONS = """# Research Workflow

Follow this workflow for all research requests:

0. **Pre-Flight**: Identify ambiguity in the user's prompt and anticipate where information gaps might exist. Generate 3-5 distinct questions and send them individually to the ask_user tool(). Do not ask generic questions. Ask questions that will drastically narrow the search scope (e.g., "Do you prioritize speed of implementation or cost-efficiency?" or "Should I limit sources to post-2023?"). Explain *why* you are asking each question.
1. **Plan**: Break down the research into focused tasks and save them using the write_todos() tool
2. **Save the request**: Use the write_file() tool to save the user's research question to `/research_request.md`
3. **Research**: Delegate research tasks to sub-agents using the task() tool - ALWAYS use sub-agents for research, never conduct research yourself
4. **Synthesize**: Review all sub-agent findings and consolidate citations (each unique URL gets one number across all findings)
5. **Write Report**: Write a comprehensive final report to `/final_report.md` (see Report Writing Guidelines below)
6. **Verify**: Read `/research_request.md` and confirm you've addressed all aspects with proper citations and structure

## Research Planning Guidelines
- Batch similar research tasks into a single TODO to minimize overhead
- For simple fact-finding questions, use 1 sub-agent
- For comparisons or multi-faceted topics, delegate to multiple parallel sub-agents
- Each sub-agent should research one specific aspect and return findings

## Report Writing Guidelines

When writing the final report to `/final_report.md`, follow these structure patterns:

**For comparisons:**
1. Introduction
2. Overview of topic A
3. Overview of topic B
4. Detailed comparison
5. Conclusion

**For lists/rankings:**
Simply list items with details - no introduction needed:
1. Item 1 with explanation
2. Item 2 with explanation
3. Item 3 with explanation

**For summaries/overviews:**
1. Overview of topic
2. Key concept 1
3. Key concept 2
4. Key concept 3
5. Conclusion

**General guidelines:**
- Use clear section headings (## for sections, ### for subsections)
- Write in paragraph form by default - be text-heavy, not just bullet points
- Do NOT use self-referential language ("I found...", "I researched...")
- Write as a professional report without meta-commentary
- Each section should be comprehensive and detailed
- Use bullet points only when listing is more appropriate than prose

**Citation format:**
- Cite sources inline using [1], [2], [3] format
- Assign each unique URL a single citation number across ALL sub-agent findings
- End report with ### Sources section listing each numbered source
- Number sources sequentially without gaps (1,2,3,4...)
- Format: [1] Source Title: URL (each on separate line for proper list rendering)
- Example:

  Some important finding [1]. Another key insight [2].

  ### Sources
  [1] AI Research Paper: https://example.com/paper
  [2] Industry Analysis: https://example.com/analysis
"""

RESEARCHER_INSTRUCTIONS = """You are a research assistant conducting research on the user's input topic. For context, today's date is {date}.

<Task>
Your job is to use tools to gather information about the user's input topic.
You can use any of the research tools provided to you to find resources that can help answer the research question.
You can call these tools in series or in parallel, your research is conducted in a tool-calling loop.
</Task>

<Available Research Tools>
You have access to two specific research tools:
1. **tavily_search**: For conducting web searches to gather information
2. **tavily_crawl**: To traverse high-value websites
3. **tavily_extract**: To extract the content of pages found via search or crawling
4. **think_tool**: For reflection and strategic planning during research
**CRITICAL: Use think_tool after each search, extraction or crawl to reflect on results and plan next steps**
</Available Research Tools>

<Instructions>
Think like a human researcher with limited time. Follow these steps:

1. **Read the question carefully** - What specific information does the user need?
2. **Start with broader searches** - Use broad, comprehensive queries first
3. **After each search, pause and assess** - Do I have enough to answer? What's still missing?
4. **Execute narrower searches as you gather information** - Fill in the gaps
5. **Stop when you can answer confidently** - Don't keep searching for perfection
</Instructions>

<Hard Limits>
**Tool Call Budgets** (Prevent excessive searching):
- **Simple queries**: Use 2-3 search tool calls maximum
- **Complex queries**: Use up to 5 search tool calls maximum
- **Always stop**: After 5 search tool calls if you cannot find the right sources

**Stop Immediately When**:
- You can answer the user's question comprehensively
- You have 3+ relevant examples/sources for the question
- Your last 2 searches returned similar information
</Hard Limits>

<Show Your Thinking>
After each search tool call, use think_tool to analyze the results:
- What key information did I find?
- What's missing?
- Do I have enough to answer the question comprehensively?
- Should I search more or provide my answer?
</Show Your Thinking>

<Final Response Format>
When providing your findings back to the orchestrator:

1. **Structure your response**: Organize findings with clear headings and detailed explanations
2. **Cite sources inline**: Use [1], [2], [3] format when referencing information from your searches
3. **Include Sources section**: End with ### Sources listing each numbered source with title and URL

Example:
```
## Key Findings

Context engineering is a critical technique for AI agents [1]. Studies show that proper context management can improve performance by 40% [2].

### Sources
[1] Context Engineering Guide: https://example.com/context-guide
[2] AI Performance Study: https://example.com/study
```

The orchestrator will consolidate citations from all sub-agents into the final report.
</Final Response Format>
"""

TASK_DESCRIPTION_PREFIX = """Delegate a task to a specialized sub-agent with isolated context. Available agents for delegation are:
{other_agents}
"""

SUBAGENT_DELEGATION_INSTRUCTIONS = """# Sub-Agent Research Coordination

Your role is to coordinate research by delegating tasks from your TODO list to specialized research sub-agents.

## Delegation Strategy

**DEFAULT: Start with 1 sub-agent** for most queries:
- "What is quantum computing?" → 1 sub-agent (general overview)
- "List the top 10 coffee shops in San Francisco" → 1 sub-agent
- "Summarize the history of the internet" → 1 sub-agent
- "Research context engineering for AI agents" → 1 sub-agent (covers all aspects)

**ONLY parallelize when the query EXPLICITLY requires comparison or has clearly independent aspects:**

**Explicit comparisons** → 1 sub-agent per element:
- "Compare OpenAI vs Anthropic vs DeepMind AI safety approaches" → 3 parallel sub-agents
- "Compare Python vs JavaScript for web development" → 2 parallel sub-agents

**Clearly separated aspects** → 1 sub-agent per aspect (use sparingly):
- "Research renewable energy adoption in Europe, Asia, and North America" → 3 parallel sub-agents (geographic separation)
- Only use this pattern when aspects cannot be covered efficiently by a single comprehensive search

## Key Principles
- **Bias towards single sub-agent**: One comprehensive research task is more token-efficient than multiple narrow ones
- **Avoid premature decomposition**: Don't break "research X" into "research X overview", "research X techniques", "research X applications" - just use 1 sub-agent for all of X
- **Parallelize only for clear comparisons**: Use multiple sub-agents when comparing distinct entities or geographically separated data

## Parallel Execution Limits
- Use at most {max_concurrent_research_units} parallel sub-agents per iteration
- Make multiple task() calls in a single response to enable parallel execution
- Each sub-agent returns findings independently

## Research Limits
- Stop after {max_researcher_iterations} delegation rounds if you haven't found adequate sources
- Stop when you have sufficient information to answer comprehensively
- Bias towards focused research over exhaustive exploration"""

In [4]:
import sys
import threading
import traceback
from datetime import datetime

import dotenv
dotenv.load_dotenv(dotenv_path='secrets.env')

from langchain.chat_models import init_chat_model
from deepagents import create_deep_agent, SubAgent
from langchain_core.runnables import RunnableConfig

from playground.tools import ask_user, tavily_crawl, tavily_extract, tavily_search


In [5]:
model="google_genai:gemini-2.5-flash"

# Limits
max_concurrent_research_units = 3
max_researcher_iterations = 3

# Get the current date
current_date = datetime.now().strftime("%Y-%m-%d")

INSTRUCTIONS = (
        RESEARCH_WORKFLOW_INSTRUCTIONS
        + "\n\n"
        + "=" * 80
        + "\n\n"
        + SUBAGENT_DELEGATION_INSTRUCTIONS.format(max_concurrent_research_units=max_concurrent_research_units,
                                                  max_researcher_iterations=max_researcher_iterations,
                                                  )
)

research_agent: SubAgent = {
    "name": "research-agent",
    "description": "Used to research in-depth questions",
    "system_prompt": RESEARCHER_INSTRUCTIONS.format(date=current_date),
    "tools": [tavily_search, tavily_crawl, tavily_extract],
    "model": model,
}

agent = create_deep_agent(
    model=init_chat_model(model),
    tools=[tavily_search, tavily_crawl, tavily_extract, ask_user],
    system_prompt=INSTRUCTIONS,
    subagents=[research_agent],
)

# a lock for interactive tools (i.e.: ask_user)
app_lock = threading.Lock()

config: RunnableConfig = {
    "configurable": {"thread_lock": app_lock},
}

result = agent.invoke({"messages": [{"role": "user", "content": "how to best plant an apple tree?"}]},
                      config)

result

question: Are you planning to plant a bare-root tree or a potted/container-grown tree? (Planting techniques can vary slightly between bare-root and potted trees, especially regarding root preparation and initial care.)
question: What is your general geographic location or climate zone? (Climate and hardiness zones significantly impact apple tree variety selection and planting times, and this information will help tailor the advice.)
question: Do you have a preference for a specific apple tree variety, or are you looking for general advice that applies to most apple trees? (Some varieties might have specific planting requirements, though general advice covers most cases. Knowing this will help me focus the research.)
question: What kind of soil do you have in your planting area (e.g., sandy, clay, loam) and what is its drainage like? (Soil type and drainage are crucial for apple tree health and will dictate whether soil amendments are necessary, as well as influencing watering schedules

{'messages': [HumanMessage(content='how to best plant an apple tree?', additional_kwargs={}, response_metadata={}, id='26ed41b3-cf50-4923-8138-14d219538168'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'ask_user', 'arguments': '{"justification": "Soil type and drainage are crucial for apple tree health and will dictate whether soil amendments are necessary, as well as influencing watering schedules.", "question": "What kind of soil do you have in your planting area (e.g., sandy, clay, loam) and what is its drainage like?"}'}, '__gemini_function_call_thought_signatures__': {'0946252e-07bf-47d3-8bf3-3d432f91a5a8': 'Cs8LAXLI2nz+o/aDMhmDezIiDMkEvl2W3gsm8JxnkOKxoVlTBUuNJknzhOCVR+D/AsSLvTF93NMp5N2S3oJzXsGNg2kP2wjleopWwweQuJt0mGGr7/k0w/o41GCcTnmzr8vLrWbRqK8VouBGWmcDJgtBbbc3LwK7nm52zhirDNKobg/fsTB6dmXoc2YDT3RhIMuVeUm4ypC8rJvGFtsORXVinhYkXMlwo1EOphoUn6c7QwQTZjxkweBq6RmtBrfEFgTAe4OlUTg+OAU9WcVuWxA1GhCEdmP7u5r6VbU2emlRnGVRE8t8sxgWtinfDJ9AGUjtLEZgEobARwny+9Qt7LGxRzcx1o79z

In [10]:
# Print the agent's response
# print('\n'.join(result['files']['/final_report.md']['content']))
from IPython.display import display, Markdown
display(Markdown('\n'.join(result['files']['/final_report.md']['content'])))

# How to Best Plant a Bare-Root Apple Tree in Coastal Washington's Clay Soil

Planting a bare-root apple tree requires careful attention to site preparation, planting technique, and initial care, especially when dealing with specific regional climates and challenging soil types like clay. This guide provides comprehensive instructions tailored for coastal Washington, incorporating best practices for bare-root trees and clay soil conditions.

## 1. Site Selection and Preparation

Selecting the right site is crucial for the long-term health and productivity of your apple tree.

### Sunlight and Drainage
Apple trees need at least 6-8 hours of direct sunlight daily [1, 5]. Choose a location that receives ample sun throughout the day. Well-drained soil is essential, as apple trees will not tolerate consistently soggy conditions, which can lead to root and crown rot [1, 4, 10]. Avoid low-lying areas where water tends to collect [1, 5].

### Addressing Clay Soil Challenges
Coastal Washington often features clay soil, which can impede drainage. To counteract this, it is highly recommended to improve drainage before planting:
*   **Planting High / Berms / Mounds:** This is a consistently recommended practice for clay soil. Build up the planting area into an elevated mound or berm to raise the root crown above the surrounding grade. This prevents water from pooling around the base of the tree and improves surface drainage [7, 9].
*   **Overall Soil Improvement:** Incorporating decomposed organic matter (DOM), such as compost, well-rotted manure, or leaf mold, into a broader area around the planting site will improve the overall soil structure and long-term drainage [7, 9]. Aim for organic material to make up 20-30% of the clay for good results [8].
*   **Avoid Amending the Planting Hole Directly:** Do not amend the immediate planting hole with significantly different soil, such as rich compost or sand. This can create a "bathtub effect" where water drains slowly from the amended area into the surrounding heavy clay, leading to root suffocation [8, 10].
*   **Scoring the Hole Sides:** When digging the planting hole, rough up or "score" the sides of the hole to prevent a smooth, compacted interface. This encourages roots to grow into the native soil and prevents water from collecting in the hole [7, 9].

## 2. Choosing the Right Apple Variety for Coastal Washington

The Puget Sound region provides sufficient chill hours for most apple varieties, but selecting disease-resistant types is important due to the prevalence of apple scab [4, 11, 12].

### Recommended Varieties
*   Boskoop [4]
*   Red BlackGold [4]
*   WhiteGold [4]
*   Lapins [4]
*   Vandalay [4]
*   Sweetheart [4]
*   Tehranivee [4]
*   Honeycrisp [4]
*   Orange Pippin [4]
*   Ashmead's Kernel [4]
*   Cortland [4]

### Varieties to Approach with Caution (due to scab susceptibility)
*   Gala [4]
*   Golden Delicious [4]
*   Red Delicious [4]
*   Granny Smith [4]

## 3. Preparing and Planting the Bare-Root Tree

### Optimal Planting Time
The best time to plant bare-root apple trees in coastal Washington is in **late winter or early spring** while the tree is dormant [1, 2].

### Root Care and Hole Preparation
1.  **Inspect and Rehydrate Roots:** Upon receiving your bare-root tree, remove packaging and inspect roots for damage, mold, or rot. Plump, firm roots indicate a healthy tree [1]. Soak the roots in a bucket of water for 1-2 hours, or up to 24 hours, before planting [1, 2, 5].
2.  **Trim Roots:** Trim off any diseased, kinked, or broken roots with sharp, sanitized pruning shears [2, 5].
3.  **Dig the Hole:** Dig a hole two to three times as wide as the root system and deep enough to accommodate the roots without bending them [1, 3, 5]. For clay soil, remember to rough up or score the sides of the hole [7, 9]. Loosen the soil at the bottom of the hole to encourage downward root growth [1].

### Planting the Tree
1.  **Position the Tree:** Create a small cone of soil in the center of the planting hole. Spread the roots evenly over this cone [6].
2.  **Graft Union Placement:** Position the tree so that the graft union (the swollen area where the scion meets the rootstock) remains 2-3 inches *above* the ground level [1, 3, 5, 10]. Burying the graft union can lead to crown rot and cause the scion to root, negating the benefits of the rootstock.
3.  **Backfill the Hole:** Gradually backfill the hole with the excavated soil (which ideally has been mixed with organic matter for overall soil improvement, not just in the hole). Water thoroughly as you fill to settle the soil around the roots and eliminate air pockets without compacting the soil [1, 5].

## 4. Post-Planting Care

### Watering
Keep the soil consistently moist but not waterlogged, especially during the first year. Provide a deep soak with a hose about twice a week to encourage root establishment [1, 3, 5].

### Staking
Staking may not be necessary for older, more established bare-root trees that stand confidently. However, younger, smaller saplings vulnerable to wind may benefit from staking. If staking, insert the stake into the ground before planting to avoid damaging the roots [3, 5].

### Initial Pruning
Initial pruning helps shape the tree and promotes strong growth [2].
*   Trim the central leader (trunk) to about knee height, or 18-24 inches from the ground, removing at least one-third of the original height [2, 5].
*   Remove any limbs that are lower than 18 inches from the ground [3, 5].
*   Cut the tips of remaining branches at an angle to encourage outward growth. Remove limbs that form tight angles with the trunk, favoring wider angles to prevent crowding as the tree fruits. Spreading tools can be used to manually widen branch angles [3, 5].
*   If the tree produces blossoms in its first year, remove them to allow the tree to focus its energy on root establishment [3, 5].

## 5. Pest and Disease Management in Coastal Washington

Homeowners are responsible for controlling horticultural pests and diseases [11]. Proactive, non-chemical methods and selecting disease-resistant varieties are recommended.

### Diseases
*   **Apple Scab (Venturia inaequalis):** This is a common and destructive fungal disease in Western Washington [12, 13].
    *   **Management:** Apply micronized sulfur (without oil) between bud development stages 6-7 (before full bloom) to prevent apple scab and powdery mildew [14]. Fungicides like Bonide Captan 50 WP can also be used [15].
*   **Powdery Mildew:** Also a common fungal disease, often managed with similar sprays as apple scab [14].

### Pests
*   **Spider Mites:** Can cause significant damage [16].
*   **Codling Moth & Apple Maggot:** These are key insect pests causing "worms in the apple" [14].
    *   **Management:**
        *   Remove and discard (do not compost) infested fallen fruit [14].
        *   Apply kaolin clay spray (e.g., Surround) at petal fall to coat trees and fruit, making them unrecognizable to pests [14].
        *   For a high-labor but effective method, bag individual fruits with nylon footies or maggot barriers after thinning. This is effective against apple maggot and 60-80% effective against codling moth if applied early [14].

### Sources
[1] Planting Bare Root Apple Trees: A Complete Guide — Raintree Nursery: https://raintreenursery.com/blogs/pnw-edible-plant-blog/planting-bare-root-apple-trees?srsltid=AfmBOoqeff3WrzA8BN2YhNpCYnmYIxKq7d8eAc3xbc3SlDzDmoKYJk7-
[2] Bare Root Essentials | Coastal Country: https://www.coastalcountry.com/resource/blog-posts/coastal-101/bare-root-essentials?srsltid=AfmBOoqFsBagGP1tA4wHMo7TTjnuj5QNB6OUjOQi1t361zLUEou_Gk1O
[3] How to Plant a Bare Root Apple Tree - Food Gardening Network: https://foodgardening.mequoda.com/articles/how-to-plant-a-bare-root-apple-tree/
[4] Recommended Fruit Trees for the Puget Sound: https://nwfruit.org/recommended-fruit-trees/
[5] Planting Bare-Root Fruit Trees | UC Master Gardener Program of Sonoma County: https://ucanr.edu/site/uc-master-gardener-program-sonoma-county/planting-bare-root-fruit-trees
[6] Planting Bare Root Apple Trees - Farmette in Bloom: https://farmetteinbloom.com/planting-bare-root-apple-trees/
[7] Dwarf Apple tree in heavy clay - Garden Stack Exchange: https://gardening.stackexchange.com/questions/21913/dwarf-apple-tree-in-heavy-clay
[8] Amending clay soil for vegetable garden and fruit trees/shrubs - Permies.com: https://permies.com/t/44641/Amending-clay-soil-vegetable-garden
[9] Improving Soil (Zone 4 Clay) - General Fruit Growing: https://growingfruit.org/t/improving-soil-zone-4-clay/46648
[10] Apple/Pear Trees in Clay, wettish soil, Vermont - Habitat Talk: https://habitat-talk.com/threads/apple-pear-trees-in-clay-wettish-soil-vermont.6510/
[11] Apple Pest Management for Home Gardeners (A2179): https://hort.extension.wisc.edu/files/2023/12/Apple-Pest-Management-for-Home-Gardeners-1.pdf
[12] Apple (Malus spp.)-Scab | Pacific Northwest Pest Management ...: https://pnwhandbooks.org/plantdisease/host-disease/apple-malus-spp-scab
[13] Apple (Malus spp.)-Scab | Pacific Northwest Pest Management ...: https://pnwhandbooks.org/plantdisease/host-disease/apple-malus-spp-scab
[14] Disease and Pest Management in the Home Orchard: https://www.cloudmountainfarmcenter.org/education/grow-tips/disease-and-pest-management-in-the-home-orchard/
[15] Apple (Malus spp.)-Scab | Pacific Northwest Pest Management ...: https://pnwhandbooks.org/plantdisease/host-disease/apple-malus-spp-scab
[16] Webbing on apple trees in Washington state can be caused by ...: https://www.facebook.com/groups/Insect.Identification/posts/1745874258861212/
